# Monte Carlo Tree Search
*Monte Carlo Tree Search* provides a way to evaluate **game state** without any **strategic knowledge** about the game. Instead of game-specific heuristics, the MCTS algorithm simulates *random games* to estimate how good a position is. One of these random games is a called: **rollout** 

*Monte Carlo Tree Search* is part of a larger family of *Monte Carlo Algorithms*, which use **Randomness** to analyze extremely complex situations. 

**The key idea** is that your **estimate** gets **more accurate** as you do **more rollouts** 

Each round of the MCTS algorithm takes three steps:
1. Add a new board position to the MCTS tree
2. Simulate a random game from that position
3. Update the tree statistics with the results of that random game

*You repeat this process as many times as you can in the time available. Then the statistics at the top of the tree tell you which move to pick* 

Every time you repeat the steps, the tree gets bigger, and the estimates at the top get more accurate. Normally, you stop after a fixed number of rounds or after a fixed amount of time passes. At that point you select the move that has the highest winning percentage.

### MCTSNode Class
This class is built to represent any node in our tree. We will track the following properties:
* ```game_state```: The current state of the game (board position & next player) at this node in the tree
* ```parent```: The parent MCTSNode that led to this one. If NONE = root of the tree
* ```move```: The last move that directly lead to this node
* ```children```: A list of all child nodes in the tree
* ```win_counts``` & ```num_rollouts```: Statistics about the rollouts that started from this node
* ```unvisited_moves```: A list of all legal moves from this position that aren't yet part of the tree. Whenever we add a new node to the tree, we pull one move out of this list and generate a new MCTSNode for it, and add it to the children list

In [10]:
import random
import math

In [ ]:
class MCTSNode(object):
    """
    This node is meant to represent any node in our tree
    """
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = game_state
        self.parent = parent
        self.move = move
        self.win_counts = {
            Player.black = 0,
            Player.white = 0
        }
        self.num_rollouts = 0
        self.children = []
        self.unvisited_moves = game_state.legal_moves()
        
    def add_random_child(self):
        """
        adding random child to the tree, from legal moves
        """
        index = random.randint(0, len(self.unvisited_moves) - 1)
        new_move = self.unvisited_moves.pop(index)
        new_game_state = self.game_state.apply_move(new_move)
        new_node = MCTSNode(new_game_state, self, new_move)
        self.children.append(new_node)
        return new_node
    
    def record_win(self, winner):
        """
        Updating wins and rollouts
        """
        self.win_counts[winner] += 1
        self.num_rollouts += 1
    
    def can_add_child(self):
        """
        This method reports whether this position has any legal moves that haven't yet been added to the tree
        """
        return len(self.unvisited_moves) > 0
    
    def is_terminal(self):
        """
        This method reports whether the game is over at this node. If so, we cannot search any further in this node
        """
        return self.game_state.is_over()
    
    def winning_frac(self, player):
        """
        This method returns the fraction of rollouts that were won by a given player (updates the deeper we go)
        """
        return float(self.win_counts[player]) / float(self.num_rollouts)

### The MCTS Algorithm
The algorithm will work as follows: 
* Start by creating a new tree. The **root node** is the current game state
* We loop for a *fixed* number of *round* for each turn, causing us to repeatedly generate **rollouts**
    * Each round begins by walking down the tree until you find a node where you can add a child (any board position that has a legal move that isn't yet in the tree)
    * ```select_move()``` hides the work of choosing the best branch to explore
    * After finding a suitable node, we call ```add_random_child()``` to pick any follow-up move and bring it into the tree. At this point, node is a newly created MCTSNode that has zero rollouts
    * We now start the rollout from this node by calling ```simulate_random_game```
    * Finally, we update the win counts of the newly created node and all it's acestors (the more rollouts, the higher accuracy)

In [12]:
class MCTSAgent():
    """
    When building in production: MCTSAgent(agent.Agent)
    Where agent.Agent is an agent playing our game. Please look at documentation in: 
    
    https://github.com/dmbernaal/SigmaGo.git 
    
    for more information
    """
    def select_move(self, game_state):
        
        # Initializing our tree with our root
        root = MCTSNode(game_state)
        
        # Will loop through number of rounds
        for i in range(self.num_rounds):
            
            # initializing our node as root
            node = root
            
            # While we can add more nodes & the game isn't over
            while (not node.can_add_child()) and (not node.is_terminal()):
                node = self.select_child(node)
                
            # Adding new child node into the tree, if we can    
            if node.can_add_child():
                node = node.add_random_child()
            
            # Simulate a random game from this node
            winner = self.simulate_random_game(node.game_state)
            
            # Propogates the score back up the tree
            while node is not None:
                node.record_win(winner)
                node = node.parent
                
        """
        Now that we have go through all rounds, we need to pick a move. We will do this by looping over all top-level branches and pick the one with the best winning percentage. 
        """
        
        best_move = None
        best_pct = -1.0
        
        # Looping through all children in our tree
        for child in root.children:
            
            # Calculate the winning pct
            child_pct = child.winning_pct(game_state.next_player)
            
            # Which child has the highest pct? Make that our best_pct
            if child_pct > best_pct:
                best_pct = child_pct
                best_move = child.move # grab that move with highest pct
        return best_move
    
    
    def select_child(self, node):
        total_rollouts = sum(child.num_rollouts for child in node.children)
        
        best_score = -1
        best_child = None
        for child in node.children:
            score = uct_score(total_rollouts, child.num_rollouts, child.winning_pct(node.game_state.next_player), self.temperature)
            
            if score > best_score:
                best_score = uct_score
                best_child = child
        
        return best_child

### Upper Condifence Bound For Trees (UCT)
We have a limited amount of time to spend on each turn, which means we can perform only a fixed number of rollouts. Given each rollout improves our evaluation of a single possible move. This makes it a *limited resource* we must choose wisely. 

**UCT** helps with choosing the right budget. 

#### Exploitation 
Our first goal is to spend our time looking for the best moves. As we complete more rollouts throughout the branches, our estimates will get more accurate. Which will remove false-positives from the list of possible moves (best moves)

For each node we are considering, we compute the winning percentage $w$ to represent the exploitation goal

#### Exploration
We want to get more accurate evaluations for the branches we have visited the least. It may be the case that our best move has a low PCT due to the fact we haven't done enough roll-out on that branch. Thus we will improve it's statistics with more roll-out 

To represent exploration, we compute where $N$ is the total number of rollouts, and $n$ is the number of rollouts that stated with the node under consideration. 

### UCT Formula
####  $$w+c\sqrt{\frac{logN}{n}}$$

* $w$ = winning percentage 
* $N$ = total number of rollouts
* $n$ = the number of rollouts that started with the node under consideration
* $c$ = a parameter that represents your preferred balance between **exploitation** and **exploration**
    * **Larger value of $c$** = spend more time visiting the least explored nodes
    * **Smaller value of $c$** = spend more time gathering a better evalauation of the most promising node

To find the best value of $c$ it's best to do *trial & error* 

The node with the highest UCT score is the start of the next rollout

You calculate the winning percentage from the point of view of the player who'd pick the next move, therefor it will alternate between players as you walk down the tree

In [13]:
# UCT Score Function
def uct_score(parent_rollouts, child_rollouts, win_pct, temperature):
    exploration = math.sqrt(math.log(parent_rollouts) / child_rollouts)
    return win_pct + temperature * exploration 